In [1]:
!pip install efficientnet_pytorch
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git
!pip install torch_optimizer

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp37-none-any.whl size=16031 sha256=9d9904df62039ef2a710e0f1f9f9e74f26b04d9b7ae87fde06ff69ca0f68584e
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-vmls68rc
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-vmls68rc
  Created wheel for torch-poly-lr-decay: filename=torch_poly_lr_decay-0.0.1-cp37-none-any.whl size=3833 sha256=93a796f866b4ac2f3d73a408020564c18390fdf7835f8453dbc13d21e277e18e
  Stored in directory: /tmp/pip-ephem-wheel-cache-axnsojc9/wheels/5a/b7/09/d748b20c9bdfc768a33c37a28b2ad7dd9ec3e79e5152cb1618
Successfully built torch-poly-lr-decay
     |████████████████████████████████| 81kB 3.8MB/s 


In [2]:
import torch
import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
# from torch_poly_lr_decay import PolynomialLRDecay
import random


torch.set_num_threads(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import torch_optimizer as optim
from torch_poly_lr_decay import PolynomialLRDecay

In [4]:
torch.cuda.is_available()

True

In [5]:
labels_df = pd.read_csv('/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/dirty_mnist_2nd_answer.csv')[:]

!mkdir "./dirty_mnist"
!unzip "/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/dirty_mnist_2nd.zip" -d "./dirty_mnist/"


imgs_dir = np.array(sorted(glob.glob('./dirty_mnist/*.png')))[:]
labels = np.array(labels_df.values[:,1:])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./dirty_mnist/45000.png  
 extracting: ./dirty_mnist/45001.png  
 extracting: ./dirty_mnist/45002.png  
 extracting: ./dirty_mnist/45003.png  
 extracting: ./dirty_mnist/45004.png  
 extracting: ./dirty_mnist/45005.png  
 extracting: ./dirty_mnist/45006.png  
 extracting: ./dirty_mnist/45007.png  
 extracting: ./dirty_mnist/45008.png  
 extracting: ./dirty_mnist/45009.png  
 extracting: ./dirty_mnist/45010.png  
 extracting: ./dirty_mnist/45011.png  
 extracting: ./dirty_mnist/45012.png  
 extracting: ./dirty_mnist/45013.png  
 extracting: ./dirty_mnist/45014.png  
 extracting: ./dirty_mnist/45015.png  
 extracting: ./dirty_mnist/45016.png  
 extracting: ./dirty_mnist/45017.png  
 extracting: ./dirty_mnist/45018.png  
 extracting: ./dirty_mnist/45019.png  
 extracting: ./dirty_mnist/45020.png  
 extracting: ./dirty_mnist/45021.png  
 extracting: ./dirty_mnist/45022.png  
 extracting: ./dirty_mnist/45023.png  
 extracting: ./dirty_mnist/4

In [6]:
len(imgs_dir)

50000

In [7]:
imgs=[]
for path in tqdm(imgs_dir[:50000]):
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    imgs.append(img)
imgs=np.array(imgs)

100%|██████████| 50000/50000 [00:43<00:00, 1148.19it/s]


In [8]:
imgs.shape

(50000, 256, 256)

In [9]:
# 저장소에서 load
class MnistDataset_v1(Dataset):
    def __init__(self, imgs_dir=None, labels=None, transform=None, train=True):
        self.imgs_dir = imgs_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get
        img = cv2.imread(self.imgs_dir[idx], cv2.IMREAD_COLOR)
        img = self.transform(img)
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img
        
        pass
    


# 메모리에서 load
class MnistDataset_v2(Dataset):
    def __init__(self, imgs=None, labels=None, transform=None, train=True):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        self.train=train
        pass
    
    def __len__(self):
        # 데이터 총 샘플 수
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # 1개 샘플 get1
        img = self.imgs[idx]
        img = self.transform(img)
        
        if self.train==True:
            label = self.labels[idx]
            return img, label
        else:
            return img

In [10]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  

In [11]:
# EfficientNet -b2(pretrained)
# MultiLabel output

class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels):
        super(EfficientNet_MultiLabel, self).__init__()
        self.network = EfficientNet.from_pretrained('efficientnet-b3', in_channels=in_channels)
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.output_layer1 = nn.Linear(1000, 160)
        self.output_layer2 = nn.Linear(160, 26)
        self.conv_bn = nn.BatchNorm2d(3)
        self.dense_bn1 = nn.BatchNorm1d(160)
        self.dense_bn2 = nn.BatchNorm1d(26)
        # self.output_layer2 = nn.Linear(500, 250)
        # self.dense2_bn = nn.BatchNorm1d(250)
        # self.output_layer3 = nn.Linear(250, 26)

    def forward(self, x):
        x = F.relu(self.conv_bn(self.conv2d(x)))
        x = F.relu((self.network(x)))
        x = torch.relu(self.dense_bn1(self.output_layer1(x)))
        x = torch.sigmoid(self.dense_bn2(self.output_layer2(x)))
        # x = torch.relu(self.output_layer2(x))
        # x = torch.sigmoid(self.output_layer3(x))
        return x

In [12]:
# 해당 코드에서는 1fold만 실행

kf = KFold(n_splits=2, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(imgs):
    folds.append((train_idx, valid_idx))

In [13]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip()

    # transforms.ToPILImage(),
    transforms.Resize((300, 300)), # 이게 필요할까
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=(-3, 3)),
    transforms.RandomVerticalFlip(p=0.5),
    # transforms.ToTensor(),
    transforms.Normalize((0.5350), (0.3008)) #  [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    ])

valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((300, 300)),
    # transforms.RandomRotation(degrees=(-3, 3)),
    transforms.Normalize((0.5350), (0.3008))
    ])

In [14]:
# # 이미지의 평균, 표준편차 구하기
# batch_size=64
# # Data Loader
# train_dataset = MnistDataset_v2(imgs = imgs[train_idx], labels=labels[train_idx], transform=train_transform)
# loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# mean = 0.0
# for images, _ in tqdm(loader):
#     batch_samples = images.size(0) 
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
# mean = mean / len(loader.dataset)

# var = 0.0
# for images, _ in tqdm(loader):
#     batch_samples = images.size(0)
#     images = images.view(batch_samples, images.size(1), -1)
#     var += ((images - mean.unsqueeze(1))**2).sum([0,2])
# std = torch.sqrt(var / (len(loader.dataset)*256*256))

# print(mean)
# print(std)

In [ ]:
seed_everything(42)

# 5개의 fold 모두 실행하려면 for문을 5번 돌리면 됩니다.
for fold in range(1):
    fold = 0 # fold 2 부터 시작
    model = EfficientNet_MultiLabel(in_channels=3).to(device)
#     model = nn.DataParallel(model)
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]





    # ●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●
    epochs=120
    batch_size=24         # 자신의 VRAM에 맞게 조절해야 OOM을 피할 수 있습니다.
    # b1
    # 64 -> die
    # 58 -> NO DIE
    # 60 -> no die
    

    # b2
    # 60 -> die
    # 40 -> no die
    # 48 -> no die & 1 epoch : 555 -> 23h


    # B2
    # 2 FOLD
    # 24 BATCH -> train 0.838 , valid 0.765

    # b2
    # 2 fold , 1000,26 linear 
    # 16 batch fold 2 에서 die

    # b5
    # 300 300 24 batch OOM
    # 18 no die too long 1415sec / epoch


    # b3
    # 32 die
    # 28
    # ●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●
    
    # Data Loader
    train_dataset = MnistDataset_v2(imgs = imgs[train_idx], labels=labels[train_idx], transform=train_transform)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = MnistDataset_v2(imgs = imgs[valid_idx], labels = labels[valid_idx], transform=valid_transform)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)       
    
    
    # optimizer
    # polynomial optimizer를 사용합니다.
    # optimizer = torch.optim.Adam(model.parameters(), lr =(1e-3)) # <= original

    optimizer = optim.RangerVA(
    model.parameters(),
    lr=1e-3,
    alpha=0.5,
    k=6,
    n_sma_threshhold=5,
    betas=(.95, 0.999),
    eps=1e-5,
    weight_decay=0,
    amsgrad=True,
    transformer='softplus',
    smooth=50,
    grad_transformer='square'
)
#     optimizer = optim.AccSGD(
#     model.parameters(),
#     lr=1e-3,
#     kappa=1000.0,
#     xi=10.0,
#     small_const=0.7,
#     weight_decay=0
# )
    

    


    decay_steps = (len(train_dataset)//batch_size)*epochs
    scheduler_poly_lr_decay = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)
    criterion = torch.nn.BCELoss()
    
    epoch_accuracy = []
    valid_accuracy = []
    valid_losses=[]
    valid_best_accuracy=0
    for epoch in range(epochs):
        model.train()
        batch_accuracy_list = []
        batch_loss_list = []
        start=time.time()
        for n, (X, y) in enumerate((train_loader)):
            X = torch.tensor(X, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            y_hat = model(X)
            
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            scheduler_poly_lr_decay.step()

            
            y_hat  = y_hat.cpu().detach().numpy()
            y_hat = y_hat>0.5
            y = y.cpu().detach().numpy()

            batch_accuracy = (y_hat == y).mean()
            batch_accuracy_list.append(batch_accuracy)
            batch_loss_list.append(loss.item())

        model.eval()
        valid_batch_accuracy=[]
        valid_batch_loss = []
        with torch.no_grad():
            for n_valid, (X_valid, y_valid) in enumerate((valid_loader)):
                X_valid = torch.tensor(X_valid, device=device)#, dtype=torch.float32) # To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True)
                y_valid = torch.tensor(y_valid, device=device, dtype=torch.float32) # To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True)
                y_valid_hat = model(X_valid)
                
                valid_loss = criterion(y_valid_hat, y_valid).item()
                
                y_valid_hat = y_valid_hat.cpu().detach().numpy()>0.5
                
                
                valid_batch_loss.append(valid_loss)
                valid_batch_accuracy.append((y_valid_hat == y_valid.cpu().detach().numpy()).mean())
                
            valid_losses.append(np.mean(valid_batch_loss))
            valid_accuracy.append(np.mean(valid_batch_accuracy))
            
        if np.mean(valid_batch_accuracy)>valid_best_accuracy:
            torch.save(model.state_dict(), '/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetb2_90epoch_diffGrad_fold{}.pt'.format(fold))
            valid_best_accuracy = np.mean(valid_batch_accuracy)
        print('fold : {}\tepoch : {:02d}\ttrain_accuracy / loss : {:.5f} / {:.5f}\tvalid_accuracy / loss : {:.5f} / {:.5f}\ttime : {:.0f}'.format(fold+1, epoch+1,
                                                                                                                                              np.mean(batch_accuracy_list),
                                                                                                                                              np.mean(batch_loss_list),
                                                                                                                                              np.mean(valid_batch_accuracy), 
                                                                                                                                              np.mean(valid_batch_loss),
                                                                                                                                              time.time()-start))

Loaded pretrained weights for efficientnet-b3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/pytorch_ranger/ranger913A.py:132: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
/usr/local/lib/python3.7/dist-packages/ip

fold : 1	epoch : 01	train_accuracy / loss : 0.50598 / 0.78711	valid_accuracy / loss : 0.51596 / 0.78725	time : 932
fold : 1	epoch : 02	train_accuracy / loss : 0.52175 / 0.74483	valid_accuracy / loss : 0.52733 / 0.74193	time : 931


In [3]:
!mkdir "./dirty_mnist_test"
!unzip "/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/test_dirty_mnist_2nd.zip" -d "./dirty_mnist_test/"

test_imgs_dir = np.array(sorted(glob.glob('./dirty_mnist_test/*')))


test_imgs=[]
for path in tqdm(test_imgs_dir):
    test_img=cv2.imread(path, cv2.IMREAD_COLOR)
    test_imgs.append(test_img)
test_imgs=np.array(test_imgs)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: ./dirty_mnist_test/50000.png  
 extracting: ./dirty_mnist_test/50001.png  
 extracting: ./dirty_mnist_test/50002.png  
 extracting: ./dirty_mnist_test/50003.png  
 extracting: ./dirty_mnist_test/50004.png  
 extracting: ./dirty_mnist_test/50005.png  
 extracting: ./dirty_mnist_test/50006.png  
 extracting: ./dirty_mnist_test/50007.png  
 extracting: ./dirty_mnist_test/50008.png  
 extracting: ./dirty_mnist_test/50009.png  
 extracting: ./dirty_mnist_test/50010.png  
 extracting: ./dirty_mnist_test/50011.png  
 extracting: ./dirty_mnist_test/50012.png  
 extracting: ./dirty_mnist_test/50013.png  
 extracting: ./dirty_mnist_test/50014.png  
 extracting: ./dirty_mnist_test/50015.png  
 extracting: ./dirty_mnist_test/50016.png  
 extracting: ./dirty_mnist_test/50017.png  
 extracting: ./dirty_mnist_test/50018.png  
 extracting: ./dirty_mnist_test/50019.png  
 extracting: ./dirty_mnist_test/50020.png  
 extracting: ./dirty_mnist_test/50021.pn

100%|██████████| 5000/5000 [00:04<00:00, 1094.14it/s]


In [9]:
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
model = EfficientNet_MultiLabel(in_channels=3).to(device)


best_model = model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt'))
# best_model = torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1-fold0.pt') # fold 0
best_models.append(best_model)


best_model = model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))# fold 1
# best_model = torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt') # fold 1
best_models.append(best_model)


best_models

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth



Loaded pretrained weights for efficientnet-b2


[<All keys matched successfully>, <All keys matched successfully>]

In [12]:
best_models

[<All keys matched successfully>, <All keys matched successfully>]

In [13]:
test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5350, 0.5350, 0.5350), (0.3008, 0.3008, 0.3008))
        ])

In [22]:
predictions_list = []
# 배치 단위로 추론
submission2 = pd.read_csv("/content/drive/MyDrive/daycon_data/월간데이콘/2차 배포/sample_submission.csv")

with torch.no_grad():
    for fold in range(1):
        model = EfficientNet_MultiLabel(in_channels=3).to(device)
        model.load_state_dict(torch.load('/content/drive/MyDrive/daycon_data/월간데이콘/EfficientNetB1_60epoch_0227_fold1.pt'))
        model.eval()

        test_dataset = MnistDataset_v2(imgs = test_imgs, transform=test_transform, train=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

        for n, X_test in enumerate(tqdm(test_loader)):
            X_test = torch.tensor(X_test, device=device, dtype=torch.float32)
            with torch.no_grad():
                model.eval()
                pred_test = model(X_test).cpu().detach().numpy()
                submission2.iloc[n*32:(n+1)*32,1:] += pred_test

  0%|          | 0/157 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
100%|██████████| 157/157 [00:13<00:00, 11.55it/s]


In [24]:
# submission.iloc[:,1:] = np.where(submission.values[:,1:]>=0.5, 1,0)
submission2.to_csv('EfficientNetB2-fold1.csv', index=False)

In [23]:
submission2

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1.000000,4.160714e-07,1.000000,0.000409,1.000000e+00,1.000000,0.000009,1.000000e+00,1.000000,0.003115,0.000063,3.685168e-05,6.742370e-03,0.990534,8.318760e-03,0.999411,1.109111e-09,0.001210,0.000002,1.000000,1.000000,0.000075,0.000065,9.989616e-01,4.086220e-07,0.049084
1,50001,0.573632,6.470272e-01,0.031682,0.142356,1.000000e+00,0.000075,0.999373,1.721620e-05,0.999986,0.075083,0.999123,8.154309e-01,3.198619e-06,0.004002,9.999897e-01,0.999973,1.696141e-03,0.782895,0.000161,0.000337,1.000000,1.000000,0.035911,8.258223e-08,1.901414e-04,0.000079
2,50002,0.000044,1.058104e-04,0.008368,0.998877,9.999840e-01,0.000445,1.000000,8.929648e-01,1.000000,0.076379,0.572995,1.303368e-05,1.064253e-03,0.000160,1.000932e-05,1.000000,1.923935e-05,0.998247,1.000000,0.015104,0.999800,0.000015,1.000000,4.293344e-06,8.856723e-01,1.000000
3,50003,0.989270,9.999774e-01,0.002007,0.659127,6.411766e-03,1.000000,0.371927,1.239821e-05,0.011564,0.153996,0.999995,9.999855e-01,9.999992e-01,0.001104,4.111533e-06,0.997721,4.637143e-01,0.000349,0.000779,0.170235,0.000056,0.284109,0.000849,1.000000e+00,9.999983e-01,0.999997
4,50004,0.224379,4.950597e-01,0.999975,0.022966,1.000000e+00,0.999997,0.000182,3.411054e-07,0.000002,0.000006,0.000008,8.239487e-05,1.000000e+00,0.011703,9.999998e-01,0.020197,1.000000e+00,0.000044,0.999995,1.000000,0.999997,0.066058,0.999998,5.951727e-06,9.287701e-01,0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.001327,2.695559e-02,1.000000,0.157930,3.526352e-04,0.010551,0.000009,1.000000e+00,0.000114,0.977908,0.676986,5.239103e-01,8.280212e-01,0.140700,9.849312e-01,0.000006,2.596159e-03,0.999868,0.000025,1.000000,0.826431,0.885558,0.999973,1.389849e-01,1.000000e+00,0.953627
4996,54996,1.000000,3.685665e-01,0.999242,0.045475,9.999999e-01,0.000084,0.000650,2.280729e-04,1.000000,1.000000,0.000001,3.331449e-07,4.905035e-07,0.958932,1.000000e+00,1.000000,9.895117e-01,0.000641,0.035425,0.000012,0.014644,0.000008,0.003778,9.999999e-01,1.098098e-06,1.000000
4997,54997,0.999967,3.129888e-03,0.000725,1.000000,2.342504e-04,0.999999,0.000002,6.633184e-05,0.000150,0.030564,1.000000,9.998972e-01,9.996374e-01,0.997326,9.999892e-01,0.999477,4.512504e-05,0.000002,0.000277,0.999131,0.999993,1.000000,0.990677,4.629958e-06,5.717273e-04,1.000000
4998,54998,0.977726,3.251634e-06,1.000000,0.025675,4.914207e-07,0.014137,0.704127,1.227333e-02,0.999858,0.999997,0.000037,1.312298e-01,1.140542e-02,0.403909,1.593264e-05,0.000019,1.000000e+00,1.000000,0.001604,0.014598,0.000057,0.002380,0.999999,1.163608e-04,1.375892e-03,0.912654


In [25]:
submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1.000000,0.000187,1.000000,0.017012,0.999999,1.000000,0.000014,1.000000e+00,0.999851,0.001634,0.000770,0.986557,0.997932,0.986481,0.001065,0.999786,0.002972,0.000094,0.000003,0.846326,1.000000,0.000057,0.014101,1.000000,0.001146,0.036573
1,50001,0.001251,0.248636,0.999991,0.000094,1.000000,0.000033,0.053738,3.952903e-02,0.999894,0.633905,1.000000,0.983649,0.000025,0.002840,0.999994,0.999990,0.001059,0.001590,0.000792,0.001700,1.000000,0.911273,0.999611,0.005957,0.080699,0.002135
2,50002,0.003285,0.000628,0.000908,0.999872,1.000000,0.073007,1.000000,7.305472e-02,1.000000,0.000861,0.222349,0.044512,0.000199,0.507153,0.000006,1.000000,0.000187,0.999923,1.000000,0.000002,0.999997,0.016819,1.000000,0.000746,0.054427,1.000000
3,50003,1.000000,0.999889,0.000999,0.998967,0.000013,0.961108,0.287080,3.684363e-04,0.002024,0.000483,0.999998,0.999977,0.999999,0.000130,0.000002,0.480075,0.001284,0.002418,0.000743,1.000000,0.000227,0.944971,0.000002,1.000000,0.999734,1.000000
4,50004,0.001705,0.000223,0.999996,0.001081,0.999999,1.000000,0.013126,1.960607e-04,0.000039,0.000223,0.000400,0.054176,1.000000,0.000453,0.999997,0.005788,0.999996,0.061760,0.999936,1.000000,0.999758,0.973970,1.000000,0.017868,0.006101,0.000064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.000804,0.999935,1.000000,0.102710,0.000040,0.050746,0.001091,1.000000e+00,0.060300,0.757709,0.083423,0.967752,0.445329,0.001115,0.999922,0.009081,0.015106,0.972846,0.005159,0.999996,0.328821,0.012433,0.999957,0.002954,1.000000,0.869026
4996,54996,0.999998,0.002303,0.999999,0.002276,1.000000,0.031000,0.009582,8.165670e-07,0.998117,0.999996,0.000355,0.009243,0.000343,0.231071,1.000000,1.000000,0.984877,0.000028,0.450245,0.825364,0.224975,0.008230,0.000942,0.996583,0.000037,1.000000
4997,54997,0.991698,0.000018,0.006052,1.000000,0.970225,1.000000,0.000893,3.826598e-04,0.140703,0.040505,0.999917,0.990274,0.989879,0.008604,1.000000,0.999990,0.000006,0.181966,0.004557,0.048823,0.999995,0.999997,0.999788,0.000023,0.001033,0.999935
4998,54998,0.954060,0.424957,1.000000,0.001296,0.001864,0.313756,0.038359,1.334733e-02,0.971326,0.999583,0.004085,0.506193,0.005842,0.326157,0.000497,0.000877,1.000000,0.967557,0.999647,0.714022,0.048579,0.035101,0.998962,0.000931,0.014615,0.000454


In [29]:
predictions_list = []
predictions_list.append(submission)
predictions_list.append(submission2)
predictions_list

[      index         a         b  ...         x         y         z
 0     50000  1.000000  0.000187  ...  1.000000  0.001146  0.036573
 1     50001  0.001251  0.248636  ...  0.005957  0.080699  0.002135
 2     50002  0.003285  0.000628  ...  0.000746  0.054427  1.000000
 3     50003  1.000000  0.999889  ...  1.000000  0.999734  1.000000
 4     50004  0.001705  0.000223  ...  0.017868  0.006101  0.000064
 ...     ...       ...       ...  ...       ...       ...       ...
 4995  54995  0.000804  0.999935  ...  0.002954  1.000000  0.869026
 4996  54996  0.999998  0.002303  ...  0.996583  0.000037  1.000000
 4997  54997  0.991698  0.000018  ...  0.000023  0.001033  0.999935
 4998  54998  0.954060  0.424957  ...  0.000931  0.014615  0.000454
 4999  54999  0.977760  0.998898  ...  0.999979  1.000000  0.999864
 
 [5000 rows x 27 columns],
       index         a             b  ...             x             y         z
 0     50000  1.000000  4.160714e-07  ...  9.989616e-01  4.086220e-07  0.04

In [35]:
df_concat = pd.concat((submission, submission2))

by_row_index = df_concat.groupby(df_concat.index)
df_means = by_row_index.mean()

df_means

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1.000000,0.000094,1.000000,0.008711,1.000000,1.000000,0.000012,1.000000,0.999926,0.002375,0.000416,0.493297,0.502337,0.988508,0.004692,0.999599,0.001486,0.000652,0.000002,0.923163,1.000000,0.000066,0.007083,0.999481,0.000573,0.042828
1,50001,0.287441,0.447831,0.515836,0.071225,1.000000,0.000054,0.526555,0.019773,0.999940,0.354494,0.999561,0.899540,0.000014,0.003421,0.999992,0.999981,0.001378,0.392243,0.000477,0.001019,1.000000,0.955636,0.517761,0.002979,0.040444,0.001107
2,50002,0.001665,0.000367,0.004638,0.999374,0.999992,0.036726,1.000000,0.483010,1.000000,0.038620,0.397672,0.022262,0.000632,0.253656,0.000008,1.000000,0.000103,0.999085,1.000000,0.007553,0.999899,0.008417,1.000000,0.000375,0.470050,1.000000
3,50003,0.994635,0.999933,0.001503,0.829047,0.003213,0.980554,0.329503,0.000190,0.006794,0.077239,0.999996,0.999981,0.999999,0.000617,0.000003,0.738898,0.232499,0.001384,0.000761,0.585118,0.000142,0.614540,0.000425,1.000000,0.999866,0.999998
4,50004,0.113042,0.247641,0.999986,0.012023,1.000000,0.999999,0.006654,0.000098,0.000021,0.000115,0.000204,0.027129,1.000000,0.006078,0.999998,0.012992,0.999998,0.030902,0.999966,1.000000,0.999878,0.520014,0.999999,0.008937,0.467436,0.000049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0.001065,0.513445,1.000000,0.130320,0.000196,0.030649,0.000550,1.000000,0.030207,0.867808,0.380204,0.745831,0.636675,0.070907,0.992427,0.004543,0.008851,0.986357,0.002592,0.999998,0.577626,0.448996,0.999965,0.070970,1.000000,0.911326
4996,54996,0.999999,0.185435,0.999621,0.023875,1.000000,0.015542,0.005116,0.000114,0.999059,0.999998,0.000178,0.004622,0.000172,0.595002,1.000000,1.000000,0.987194,0.000334,0.242835,0.412688,0.119809,0.004119,0.002360,0.998291,0.000019,1.000000
4997,54997,0.995832,0.001574,0.003388,1.000000,0.485230,1.000000,0.000447,0.000224,0.070427,0.035535,0.999959,0.995086,0.994758,0.502965,0.999994,0.999734,0.000025,0.090984,0.002417,0.523977,0.999994,0.999999,0.995233,0.000014,0.000803,0.999967
4998,54998,0.965893,0.212480,1.000000,0.013486,0.000932,0.163947,0.371243,0.012810,0.985592,0.999790,0.002061,0.318711,0.008624,0.365033,0.000257,0.000448,1.000000,0.983779,0.500625,0.364310,0.024318,0.018741,0.999480,0.000524,0.007996,0.456554


In [36]:
# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (df_means > 0.5) * 1
predictions_mean

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0
2,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,1
3,1,1,1,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,1,1,1
4,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,1
4996,1,1,0,1,0,1,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,1
4997,1,1,0,0,1,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,1,0,0,1
4998,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0


In [38]:
predictions_mean.iloc[:,0] = df_means.iloc[:,0]
predictions_mean

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,0,0,0,1,1,0,0,1,0,0
1,50001,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0
2,50002,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,1
3,50003,1,1,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,1,1,1
4,50004,0,0,1,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,1
4996,54996,1,0,1,0,1,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,1
4997,54997,1,0,0,1,0,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,1,0,0,1
4998,54998,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0


In [39]:
predictions_mean.to_csv("eff_net_b2_epoch60.csv", index = False)

In [ ]:
# work to do : SAM optim